# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Nov 3 2022 10:51AM,249963,19,ADV80007339,"AdvaGen Pharma, Ltd",Released
1,Nov 3 2022 10:47AM,249962,10,MSP215465,"Methapharm, Inc.",Released
2,Nov 3 2022 10:47AM,249962,10,MSP215466,"Methapharm, Inc.",Released
3,Nov 3 2022 10:47AM,249962,10,MSP215467,"Methapharm, Inc.",Released
4,Nov 3 2022 10:47AM,249962,10,MSP215468,"Methapharm, Inc.",Released
5,Nov 3 2022 10:47AM,249962,10,MSP215469,"Methapharm, Inc.",Released
6,Nov 3 2022 10:47AM,249962,10,MSP215470,"Methapharm, Inc.",Released
7,Nov 3 2022 10:47AM,249962,10,MSP215471,"Methapharm, Inc.",Released
8,Nov 3 2022 10:47AM,249962,10,MSP215474,"Methapharm, Inc.",Released
9,Nov 3 2022 10:47AM,249962,10,MSP215475,"Methapharm, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
29,249958,Released,1
30,249959,Released,16
31,249960,Released,1
32,249962,Released,9
33,249963,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
249958,NaN,NaN,1.0
249959,NaN,NaN,16.0
249960,NaN,NaN,1.0
249962,NaN,NaN,9.0
249963,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
249881,0.0,0.0,1.0
249884,0.0,0.0,1.0
249885,0.0,0.0,1.0
249887,0.0,0.0,1.0
249888,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
249881,0,0,1
249884,0,0,1
249885,0,0,1
249887,0,0,1
249888,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,249881,0,0,1
1,249884,0,0,1
2,249885,0,0,1
3,249887,0,0,1
4,249888,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,249881,,,1
1,249884,,,1
2,249885,,,1
3,249887,,,1
4,249888,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Nov 3 2022 10:51AM,249963,19,"AdvaGen Pharma, Ltd"
1,Nov 3 2022 10:47AM,249962,10,"Methapharm, Inc."
10,Nov 3 2022 10:43AM,249960,10,"Methapharm, Inc."
11,Nov 3 2022 10:40AM,249959,10,Eywa Pharma Inc.
27,Nov 3 2022 10:17AM,249958,16,"Emersa Waterbox, LLC"
28,Nov 3 2022 10:05AM,249952,15,"Virtus Pharmaceuticals, LLC"
31,Nov 3 2022 9:40AM,249956,19,"NAPP Technologies, LLC"
32,Nov 3 2022 9:40AM,249950,15,"Alliance Pharma, Inc."
77,Nov 3 2022 9:35AM,249955,19,"GCH Granules USA, Inc."
78,Nov 3 2022 9:31AM,249953,10,Bio-PRF


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Nov 3 2022 10:51AM,249963,19,"AdvaGen Pharma, Ltd",,,1
1,Nov 3 2022 10:47AM,249962,10,"Methapharm, Inc.",,,9
2,Nov 3 2022 10:43AM,249960,10,"Methapharm, Inc.",,,1
3,Nov 3 2022 10:40AM,249959,10,Eywa Pharma Inc.,,,16
4,Nov 3 2022 10:17AM,249958,16,"Emersa Waterbox, LLC",,,1
5,Nov 3 2022 10:05AM,249952,15,"Virtus Pharmaceuticals, LLC",,1,2
6,Nov 3 2022 9:40AM,249956,19,"NAPP Technologies, LLC",,,1
7,Nov 3 2022 9:40AM,249950,15,"Alliance Pharma, Inc.",,17,28
8,Nov 3 2022 9:35AM,249955,19,"GCH Granules USA, Inc.",,1,
9,Nov 3 2022 9:31AM,249953,10,Bio-PRF,,13,2


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Nov 3 2022 10:51AM,249963,19,"AdvaGen Pharma, Ltd",1,,
1,Nov 3 2022 10:47AM,249962,10,"Methapharm, Inc.",9,,
2,Nov 3 2022 10:43AM,249960,10,"Methapharm, Inc.",1,,
3,Nov 3 2022 10:40AM,249959,10,Eywa Pharma Inc.,16,,
4,Nov 3 2022 10:17AM,249958,16,"Emersa Waterbox, LLC",1,,
5,Nov 3 2022 10:05AM,249952,15,"Virtus Pharmaceuticals, LLC",2,1,
6,Nov 3 2022 9:40AM,249956,19,"NAPP Technologies, LLC",1,,
7,Nov 3 2022 9:40AM,249950,15,"Alliance Pharma, Inc.",28,17,
8,Nov 3 2022 9:35AM,249955,19,"GCH Granules USA, Inc.",,1,
9,Nov 3 2022 9:31AM,249953,10,Bio-PRF,2,13,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Nov 3 2022 10:51AM,249963,19,"AdvaGen Pharma, Ltd",1,,
1,Nov 3 2022 10:47AM,249962,10,"Methapharm, Inc.",9,,
2,Nov 3 2022 10:43AM,249960,10,"Methapharm, Inc.",1,,
3,Nov 3 2022 10:40AM,249959,10,Eywa Pharma Inc.,16,,
4,Nov 3 2022 10:17AM,249958,16,"Emersa Waterbox, LLC",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Nov 3 2022 10:51AM,249963,19,"AdvaGen Pharma, Ltd",1.0,NaN,NaN
1,Nov 3 2022 10:47AM,249962,10,"Methapharm, Inc.",9.0,NaN,NaN
2,Nov 3 2022 10:43AM,249960,10,"Methapharm, Inc.",1.0,NaN,NaN
3,Nov 3 2022 10:40AM,249959,10,Eywa Pharma Inc.,16.0,NaN,NaN
4,Nov 3 2022 10:17AM,249958,16,"Emersa Waterbox, LLC",1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Nov 3 2022 10:51AM,249963,19,"AdvaGen Pharma, Ltd",1.0,0.0,0.0
1,Nov 3 2022 10:47AM,249962,10,"Methapharm, Inc.",9.0,0.0,0.0
2,Nov 3 2022 10:43AM,249960,10,"Methapharm, Inc.",1.0,0.0,0.0
3,Nov 3 2022 10:40AM,249959,10,Eywa Pharma Inc.,16.0,0.0,0.0
4,Nov 3 2022 10:17AM,249958,16,"Emersa Waterbox, LLC",1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,999834,28.0,13.0,0.0
15,999786,50.0,28.0,0.0
16,749730,3.0,0.0,0.0
19,1499666,19.0,16.0,8.0
21,2748829,11.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,999834,28.0,13.0,0.0
1,15,999786,50.0,28.0,0.0
2,16,749730,3.0,0.0,0.0
3,19,1499666,19.0,16.0,8.0
4,21,2748829,11.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,28.0,13.0,0.0
1,15,50.0,28.0,0.0
2,16,3.0,0.0,0.0
3,19,19.0,16.0,8.0
4,21,11.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,28.0
1,15,Released,50.0
2,16,Released,3.0
3,19,Released,19.0
4,21,Released,11.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,19,21
Status,,,,,
Completed,0.0,0.0,0.0,8.0,0.0
Executing,13.0,28.0,0.0,16.0,0.0
Released,28.0,50.0,3.0,19.0,11.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,19,21
0,Completed,0.0,0.0,0.0,8.0,0.0
1,Executing,13.0,28.0,0.0,16.0,0.0
2,Released,28.0,50.0,3.0,19.0,11.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,19,21
0,Completed,0.0,0.0,0.0,8.0,0.0
1,Executing,13.0,28.0,0.0,16.0,0.0
2,Released,28.0,50.0,3.0,19.0,11.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()